# <font color="darkblue"> Prática 04: Random Forest </font>

**Objetivos:**


*   Comparar os resultados da Árvore de Decisão induzida por dados com o algoritmo de *Random Forest*;
*   Implementar o *tunning* do algoritmo de *Random Forest*.  

**Requisitos de execução:**


*   Upload do arquivo *diabetes.csv*

**Atividade 1:**

1. Visitar a base de dados: https://www.kaggle.com/uciml/pima-indians-diabetes-database
2. Carregar os dados do arquivo *diabetes.csv* utilizando o pandas.

    

In [ ]:
import pandas as pd 

diabete = pd.read_csv("diabetes.csv", sep=',')

# Pega o cabecalho do arquivo
print(diabete.head())

   Pregnancies  Glucose  BloodPressure  ...  DiabetesPedigreeFunction  Age  Outcome
0            6      148             72  ...                     0.627   50        1
1            1       85             66  ...                     0.351   31        0
2            8      183             64  ...                     0.672   32        1
3            1       89             66  ...                     0.167   21        0
4            0      137             40  ...                     2.288   33        1

[5 rows x 9 columns]


**Atividade 2:**

1. Extrair os valores do *DataFrame* pandas e dividir en dados de treino e teste.


In [ ]:
from sklearn.model_selection import train_test_split

Features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',  'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X = diabete[Features].values
y = diabete.Outcome.values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2698)

print("Tamanho treino: " + str(len(x_train)))
print("Tamanho teste: " + str(len(x_test)))


Tamanho treino: 614
Tamanho teste: 154


**Atividade 3:**

1. Utilize a classe *DecisionTreeClassifier*, importada do pacote *sklearn.tree*, para inferir aprendizado dos dados de treinamento;
2. Compute as métricas de aprendizado sobre os dados de teste.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

print('Ein: %0.4f' % (1 - accuracy_score(y_train, clf.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, clf.predict(x_test))))
print(classification_report(y_test, clf.predict(x_test)))

Ein: 0.0000
Eout: 0.3052
              precision    recall  f1-score   support

           0       0.74      0.80      0.77        98
           1       0.59      0.52      0.55        56

    accuracy                           0.69       154
   macro avg       0.67      0.66      0.66       154
weighted avg       0.69      0.69      0.69       154



**Atividade 4:**


1. Utilize a classe *RandomForestClassifier*, importada do pacote *sklearn.ensemble*, para inferir aprendizado dos dados de treinamento;
2. Compute as métricas de aprendizado sobre os dados de teste e compare com os resultados da atividade anterior.

Parâmetros:


*   *n_estimators* : número de árvore aleatórias inferidas
*   *max_depth* : profundidade máxima da árvore



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100, max_depth = 30, random_state=42)

rf.fit(x_train, y_train)

print('Ein: %0.4f' % (1 - accuracy_score(y_train, rf.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, rf.predict(x_test))))
print(classification_report(y_test, rf.predict(x_test)))

Ein: 0.0000
Eout: 0.2727
              precision    recall  f1-score   support

           0       0.75      0.87      0.80        98
           1       0.68      0.48      0.56        56

    accuracy                           0.73       154
   macro avg       0.71      0.67      0.68       154
weighted avg       0.72      0.73      0.71       154



**Atividade 5:**

1. Implemente os valores dos hiperparâmetros que serão combinados durantea a fase de validação

Parâmetros:

*   *max_features* : número de características a serem consideradas em cada divisão da árvore



In [ ]:
import numpy as np
from pprint import pprint

n_estimators = [int(x) for x in np.linspace(start=20, stop = 70, num=6)]
max_features = ['sqrt', 'log2', 8]
max_depth = [int(x) for x in np.linspace(start=10, stop = 40, num=4)]
max_depth.append(None)

param_grid = {
    'n_estimators' : n_estimators,
    'max_features' : max_features,
    'max_depth' : max_depth
}

pprint(param_grid)

{'max_depth': [10, 20, 30, 40, None],
 'max_features': ['sqrt', 'log2', 8],
 'n_estimators': [20, 30, 40, 50, 60, 70]}


Atividade 6:

1. Implementar a validação da Random Forest através da classe *GridSearchCV* do pacote *sklearn.model_selection*;
2. Imprimir os parâmetros do melhor classificador;
3. Imprimir os $E_{in}$ e $E_{out}$  e as métricas de aprendizado.

Parâmetros:


*   *estimator* : instância do classificador cujos hiperparâmetros serão analisados;
*   *cv* : número de divisões do conjunto de treinamento para ser usado na técnica de validação cruzada (10 é um bom valor observado na prática);
*   *param_grid* : conjunto de parâmetros a serem combinados durante a fase de validação.



In [ ]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()

cv_rf = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, verbose=2, n_jobs=-1)

cv_rf.fit(x_train, y_train)

print('Ein: %0.4f' % (1 - accuracy_score(y_train, cv_rf.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, cv_rf.predict(x_test))))
print(classification_report(y_test, cv_rf.predict(x_test)))

print(cv_rf.best_estimator_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:   22.9s


Ein: 0.0000
Eout: 0.2792
              precision    recall  f1-score   support

           0       0.75      0.85      0.79        98
           1       0.65      0.50      0.57        56

    accuracy                           0.72       154
   macro avg       0.70      0.67      0.68       154
weighted avg       0.71      0.72      0.71       154

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=40, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=70,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   33.9s finished


**Atividade 7:**

1. Implementar a validação da Random Forest através da classe *RandomizedSearchCV* do pacote *sklearn.model_selection*;
2. Imprimir os parâmetros do melhor classificador;
3. Imprimir os $E_{in}$ e $E_{out}$ e as métricas de aprendizado.

Parâmetros:


*   *n_iter* : número aleatório de combinações dos parâmetros que serão validadas.



In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier()

cv_rf = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, n_iter = 225, cv = 5, verbose=2, n_jobs=-1)

cv_rf.fit(x_train, y_train)

print(cv_rf.best_estimator_)

print('Ein: %0.4f' % (1 - accuracy_score(y_train, cv_rf.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, cv_rf.predict(x_test))))
print(classification_report(y_test, cv_rf.predict(x_test)))

Fitting 5 folds for each of 90 candidates, totalling 450 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 90 is smaller than n_iter=225. Running 90 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   12.3s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Ein: 0.0195
Eout: 0.3052
              precision    recall  f1-score   support

           0       0.71      0.87      0.78        98
           1       0.63      0.39      0.48        56

    accuracy                           0.69       154
   macro avg       0.67      0.63      0.63       154
weighted avg       0.68      0.69      0.67       154



[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   35.0s finished
